In [ ]:
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from gluonts.torch import DeepAREstimator
import numpy as np
from fusiontimeseries.finetuning.preprocessing.utils import get_valid_flux_traces

In [ ]:
flux_data: dict[int, np.ndarray] = get_valid_flux_traces()

records = []
for item_id, flux_trace in flux_data.items():
    for t in range(flux_trace.shape[0]):
        records.append(
            {
                "item_id": item_id,
                "time_idx": t,
                "target": flux_trace[t],
            }
        )
df = pd.DataFrame(records)

df = df.sort_values(["item_id", "time_idx"])

df["timestamp"] = pd.to_datetime("2000-01-01") + pd.to_timedelta(
    df["time_idx"], unit="h"
)
df.head()

In [ ]:
dataset = PandasDataset.from_long_dataframe(
    df,
    item_id="item_id",
    timestamp="timestamp",
    target="target",
    freq="h",  # frequency of the generated time index
)

In [ ]:
# Split the data for training and testing
training_data, test_gen = split(dataset, offset=-36)
test_data = test_gen.generate_instances(prediction_length=12, windows=3)

# Train the model and make predictions
model = DeepAREstimator(
    prediction_length=64, freq="h", trainer_kwargs={"max_epochs": 7}
).train(training_data)

forecasts = list(model.predict(test_data.input))

In [ ]:
forecasts[0].plot

In [ ]:
forecasts[0].plot()
df[df["item_id"] == 0]["target"].plot()

In [ ]:
print(model)